In [1]:
from astroquery.vizier import Vizier

from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time
import astropy.units as u

import pandas as pd
import numpy as np

import datetime
import pickle

from IPython.display import Markdown, HTML

## User Input

In [ ]:
# Specify the catalog identifier (the bibcode)
#catalog_id = "J/A+A/686/A174"  # This is the VizieR catalog ID for the paper

# Statistical analysis of asteroseismic indices and stellar parameters of TESS-observed δ Scuti stars

catalog_id = "II/366/catv2021" 
# ASAS-SN catalogue of variable stars VI: an all-sky sample of δ Scuti stars (Jayasinghe et al. 2019)

## Get the Catalog Table

In [ ]:
# Initialize the Vizier object
vizier = Vizier(row_limit=-1)

# Retrieve the table(s)
tables = vizier.get_catalogs(catalog_id)

# Check if any tables were found
if tables:
    # The 'tables' object is an AstroPy TableList.
    # You can access individual tables by index (e.g., tables[0])
    # or by the table name if available.

    # Print the first table to inspect its contents
    #print(tables[0])

    # You can access the data as an AstroPy Table object
    data_table = tables[0]
    df = data_table.to_pandas()
else:
    print(f"No tables found for catalog ID: {catalog_id}")
    
df[0:5]

In [ ]:
#ra_list  = df["RAJ2000"].astype(str).tolist()
#dec_list = df["DEJ2000"].astype(str).tolist()
coords   = SkyCoord(ra=df["RAJ2000"], dec=df["DEJ2000"], unit=(u.degree,u.degree))

#ra_in_hours = coords.ra.to(u.hour)



# # try:
# 
# # except ValueError:
# #     print('RA (in hourangle) and DEC (in deg) already exist in Landolt catalogue database')
    


In [ ]:
#df.insert(2, 'RA_hourangle', coords.ra.to(u.hourangle))
#df.insert(3, 'DEC_deg', coords.dec.value)
df['coords'] = coords

### Selects stars with User Criteria

In [ ]:
len(df)

In [ ]:
with open('file.pkl', 'wb') as file:
    pickle.dump(df, file)

In [3]:
with open('file.pkl', 'rb') as file:
    df = pickle.load(file)

In [15]:
# Location
latitude = -6.824631  * u.deg  # Latitude of Bosscha Obs
longitude = 107.617111 * u.deg
elevation = 1100 * u.m 
earth_location = EarthLocation(lat=latitude, lon=longitude , height=elevation)

now_utc = Time(datetime.datetime.utcnow(), scale='utc', location=earth_location)
lst = now_utc.sidereal_time('apparent')
lst_hour = round(lst.hour,0)

#print(f"LST saat ini = {lst} ")

lst_hms_string_angle = lst.to_string(unit=u.hour, sep=':', precision=2, pad=True)

# Method 2: Accessing the hour, minute, second components directly
# This is useful if you want to format it manually or use the components
lst_hour_component = int(lst.hour)
lst_minute_component = int((lst.hour * 60) % 60)
lst_second_component = (lst.hour * 3600) % 60

# Format manually to hms
lst_hms_string_manual = f"{lst_hour_component:02d}:{lst_minute_component:02d}:{lst_second_component:05.2f}"


print(f"Current LST: {lst}")

Current LST: 18.24268167370102 hourangle


In [8]:
df.columns

Index(['ASASSN-V', 'RAJ2000', 'DEJ2000', 'Vmag', 'Amp', 'Per', 'Type', 'Prob',
       'HJD', 'Plx', 'e_Plx', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'Vt', 'Dist',
       'WISEA', 'J-K', 'W1-W2', 'APASS', 'B-V', 'E_B-V_', 'Ref', 'GaiaDR3',
       'GALEX', 'TIC', 'pm', 'URL', '_2M', 'coords'],
      dtype='object')

In [34]:
# print(f"lst_hour = {lst_hour}")
columns       = ["ASASSN-V",'RAJ2000', 'DEJ2000', "Type", 'Vmag', "Per", "Amp", "coords"]

#type_mask = (df['Type'] == "HADS") | (df['Type'] == "DSCT")     
v_mask  = (df['Vmag'] >= 11) & (df['Vmag'] <= 13)
RA_mask = (df['RAJ2000'] >= ((lst.value - 2)*15)) & (df['RAJ2000'] <= ((lst.value + 2)*15))
Per_mask = (df['Per'] <= (4/24)) 
Amp_mask = (df['Amp'] >= 0.5)
all_mask = Amp_mask & v_mask & RA_mask & Per_mask


#display(Markdown(f"### List Bintang 11 <= V <= 13 dengan {lst_hour-2} < RA(J2000) < {lst_hour+2}"))

markdown_table = df[all_mask][columns]#.to_markdown(index=False)
#display(Markdown(markdown_table))
markdown_table    

,ASASSN-V,RAJ2000,DEJ2000,Type,Vmag,Per,Amp,coords
41944,J200315.60+585735.3,300.81500,58.95980,DSCT:,12.93,0.134870,0.84,"<SkyCoord (ICRS): (ra, dec) in deg\n (300.8..."
98597,J174006.02-263111.3,265.02507,-26.51980,EA,12.05,0.062368,1.11,"<SkyCoord (ICRS): (ra, dec) in deg\n (265.0..."
203660,J200315.55+585717.2,300.81478,58.95478,SXPHE,11.84,0.134870,0.77,"<SkyCoord (ICRS): (ra, dec) in deg\n (300.8..."
205873,J185654.40-394650.7,284.22684,-39.78093,HADS,12.85,0.078426,0.66,"<SkyCoord (ICRS): (ra, dec) in deg\n (284.2..."
326123,J183206.33+403556.4,278.02636,40.59901,HADS,12.36,0.102150,0.56,"<SkyCoord (ICRS): (ra, dec) in deg\n (278.0..."
340097,J185748.47-073440.0,284.45195,-7.57779,EA,12.83,0.119790,0.96,"<SkyCoord (ICRS): (ra, dec) in deg\n (284.4..."
549189,J173550.86-530447.4,263.96190,-53.07984,HADS,12.40,0.113380,0.53,"<SkyCoord (ICRS): (ra, dec) in deg\n (263.9..."


In [35]:
markdown_table['HA'] = lst.value - (markdown_table["RAJ2000"])/15

RA_hms = [coords.ra.hms for coords in markdown_table["coords"] ]
DE_dms = [coords.dec.dms for coords in markdown_table["coords"] ]

markdown_table["RA_hms"] = RA_hms
markdown_table["DE_dms"] = DE_dms
# for coords in markdown_table["coords"]:
#     RA_hms.append()

# markdown_table['RA_hms'] = markdown_table["coords"].ra.hms
# ra_hour_component = int(markdown_table["coords"])
# ra_minute_component = int((markdown_table["coords"] * 60) % 60)
# ra_second_component = (markdown_table["coords"] * 3600) % 60

In [36]:
markdown_table

,ASASSN-V,RAJ2000,DEJ2000,Type,Vmag,Per,Amp,coords,HA,RA_hms,DE_dms
41944,J200315.60+585735.3,300.81500,58.95980,DSCT:,12.93,0.134870,0.84,"<SkyCoord (ICRS): (ra, dec) in deg\n (300.8...",-1.811652,"(20.0, 3.0, 15.600000000008833)","(58.0, 57.0, 35.28000000000475)"
98597,J174006.02-263111.3,265.02507,-26.51980,EA,12.05,0.062368,1.11,"<SkyCoord (ICRS): (ra, dec) in deg\n (265.0...",0.574344,"(17.0, 40.0, 6.016800000020339)","(-26.0, -31.0, -11.280000000000143)"
203660,J200315.55+585717.2,300.81478,58.95478,SXPHE,11.84,0.134870,0.77,"<SkyCoord (ICRS): (ra, dec) in deg\n (300.8...",-1.811637,"(20.0, 3.0, 15.547200000014243)","(58.0, 57.0, 17.207999999998265)"
205873,J185654.40-394650.7,284.22684,-39.78093,HADS,12.85,0.078426,0.66,"<SkyCoord (ICRS): (ra, dec) in deg\n (284.2...",-0.705774,"(18.0, 56.0, 54.441600000013466)","(-39.0, -46.0, -51.34799999999245)"
326123,J183206.33+403556.4,278.02636,40.59901,HADS,12.36,0.102150,0.56,"<SkyCoord (ICRS): (ra, dec) in deg\n (278.0...",-0.292409,"(18.0, 32.0, 6.326400000020556)","(40.0, 35.0, 56.43599999999935)"
340097,J185748.47-073440.0,284.45195,-7.57779,EA,12.83,0.119790,0.96,"<SkyCoord (ICRS): (ra, dec) in deg\n (284.4...",-0.720782,"(18.0, 57.0, 48.46800000001281)","(-7.0, -34.0, -40.04400000000089)"
549189,J173550.86-530447.4,263.96190,-53.07984,HADS,12.40,0.113380,0.53,"<SkyCoord (ICRS): (ra, dec) in deg\n (263.9...",0.645222,"(17.0, 35.0, 50.85600000001875)","(-53.0, -4.0, -47.42399999999009)"
